### Reward in all, easy, and hard sessions
- Comparing Unstruc and Struc where the difference in probability is more than 0.4 

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects

exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess
trial_filter = dict(min_trials=100, clip_max=100)

rwd_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    task = exp.b2a.filter_by_trials(**trial_filter)
    mab_easy = task.filter_by_deltaprob(delta_min=0.4)
    mab_hard = task.filter_by_deltaprob(delta_min=0, delta_max=0.35)

    rp = task.get_reward_probability()
    rp_easy = mab_easy.get_reward_probability()
    rp_hard = mab_hard.get_reward_probability()

    df = pd.DataFrame(
        dict(
            trial_id=np.arange(rp.size) + 1,
            rp=rp,
            rp_easy=rp_easy,
            rp_hard=rp_hard,
            name=exp.sub_name,
            grp="struc" if task.is_structured else "unstruc",
            first_experience=True if "Exp1" in exp.sub_name else False,
        )
    )
    rwd_df.append(df)

rwd_df = pd.concat(rwd_df, ignore_index=True)
mab_subjects.GroupData().save(rwd_df, "reward_prob_difficulty_level")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statplotannot.plots.colormaps import colors_mab
from statplotannot.plots import SeabornPlotter

fig = plotting.Fig(6, 3, num=1, fontsize=10)

grpdata = mab_subjects.GroupData()
df = grpdata.reward_prob_difficulty_level
df = df[df["first_experience"] == True]

ax = fig.subplot(fig.gs[0])
hue_order = ["unstruc", "struc"]
linestyles = ["-", "--", ":"]

for i, y in enumerate(["rp", "rp_easy", "rp_hard"]):

    plot_kw = dict(data=df, x="trial_id", y=y, hue="grp", hue_order=hue_order, ax=ax)
    sns.lineplot(
        palette=colors_mab(),
        # palette=["#E89317", "#176CE8"],
        # palette=["#d58915", "#1660d0"],
        lw=1.1,
        # edgecolor="white",
        # facecolor=(0, 0, 0, 0),
        # alpha=0.4,
        ls=linestyles[i],
        err_kws=dict(edgecolor="none"),
        errorbar="se",
        **plot_kw,
    )
SeabornPlotter.fix_legend(ax)

ax.set_title(r"Performance in all sessions")
ax.set_ylabel("Performance")
# ax.set_ylim(0.45, 1)
ax.set_xticks([1, 50, 100])
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

# ax.legend("")

### Reward rate matrix
- Bins are represented by probability combination

In [ ]:
import pandas as pd
import mab_subjects

exps = mab_subjects.mostly_unstruc.allsess + mab_subjects.mostly_struc.allsess

prob_perf_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    task = exp.b2a.filter_by_trials(min_trials=100, clip_max=100)
    # print(task.probs_corr)
    # task.auto_block_window_ids()
    # mask = task.block_ids == 2
    # task = task._filtered(mask).filter_by_trials(100, 100)

    perf_mat, unique_probs = task.get_performance_prob_grid(
        n_last_trials=99, performance_metric="reward_rate"
    )

    df = pd.DataFrame(
        {
            "probs": [unique_probs],
            "perf_mat": [perf_mat],
            "name": exp.sub_name,
            "grp": exp.group_tag,
            "lesion": exp.lesion_tag,
            "dataset": exp.data_tag,
        }
    )
    df["name"] = exp.sub_name
    df["grp"] = exp.group_tag

    prob_perf_df.append(df)

prob_perf_df = pd.concat(prob_perf_df, ignore_index=True)
mab_subjects.GroupData().save(prob_perf_df, "reward_probability_matrix")

In [ ]:
import matplotlib.pyplot as plt
from neuropy.plotting import Fig
import mab_subjects
import numpy as np
from matplotlib.colors import BoundaryNorm
import matplotlib.ticker as ticker


bounds = np.linspace(0, 1, 11)
norm = BoundaryNorm(bounds, ncolors=256)

fig = Fig(6, 3, size=(8.5, 11), fontsize=12)

df = mab_subjects.GroupData().reward_probability_matrix.latest
# df = df[df["lesion"] == "pre_lesion"]
perf_both = []
for g, grp in enumerate(["struc", "unstruc"]):
    df_grp = df[df["grp"] == grp]
    perf_mean = df_grp["perf_mat"].mean()
    # perf_mean = gaussian_filter(perf_mean, sigma=0.1)
    # perf_mean = np.tril(perf_mean, k=0)
    mask = np.triu(np.ones_like(perf_mean, dtype=bool), k=0)

    perf_mean[mask] = np.nan
    perf_both.append(perf_mean)

    ax = fig.subplot(fig.gs[g])

    im = ax.pcolormesh(
        perf_mean.T,
        # cmap=GrayC_7.mpl_colormap,
        cmap="turbo",
        shading="auto",
        # vmin=0.6,
        # vmax=1,
        norm=norm,
    )
    ticks = np.arange(0, 6) + 0.5
    ax.set_xticks(ticks, [20, 30, 40, 60, 70, 80])
    ax.set_yticks(ticks, [20, 30, 40, 60, 70, 80])
    ax.set_xlim(1, 6)
    ax.set_ylim(0, 5)
    ax.spines["right"].set_visible(True)
    ax.spines["top"].set_visible(True)
    ax.set_xlabel("Higher reward probability")
    ax.set_ylabel("Lower reward probability")

    cb = plt.colorbar(im, ax=ax, shrink=0.7)
    cb.set_label("Reward rate")
    formatter = ticker.FormatStrFormatter("%.2f")
    cb.ax.yaxis.set_major_formatter(formatter)
    ax.set_title(f"{grp} sessions\n(mean of last 99 trials)")


ax = fig.subplot(fig.gs[2])
perf_diff = perf_both[0] - perf_both[1]
im = ax.pcolormesh(
    perf_diff.T,
    # cmap=GrayC_7.mpl_colormap,
    cmap="bwr",
    shading="auto",
    # vmin=0.6,
    # vmax=1,
    # norm=norm,
)
ticks = np.arange(0, 6) + 0.5
ax.set_xticks(ticks, [20, 30, 40, 60, 70, 80])
ax.set_yticks(ticks, [20, 30, 40, 60, 70, 80])
ax.set_xlim(1, 6)
ax.set_ylim(0, 5)
ax.spines["right"].set_visible(True)
ax.spines["top"].set_visible(True)
ax.set_xlabel("Higher reward probability")
ax.set_ylabel("Lower reward probability")

cb = plt.colorbar(im, ax=ax, shrink=0.7)
cb.set_label("Reward diff")
formatter = ticker.FormatStrFormatter("%.2f")
cb.ax.yaxis.set_major_formatter(formatter)
ax.set_title("Difference (struc - unstruc)")

### Reward rate blockwise

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects

exps = mab_subjects.mostly_unstruc.allsess + mab_subjects.mostly_struc.allsess

rwd_df = []
for i, exp in enumerate(exps):

    print(exp.sub_name)
    task = exp.b2a.filter_by_trials(100, 100)
    task.auto_block_window_ids()

    mask_block1 = task.block_ids == 1
    task_block1 = task._filtered(mask_block1)

    mask_block2 = task.block_ids == 2
    task_block2 = task._filtered(mask_block2)

    rp = task.get_reward_probability()
    rp_block1 = task_block1.get_reward_probability()
    rp_block2 = task_block2.get_reward_probability()

    trial_id = np.arange(rp.size) + 1
    df = pd.DataFrame(
        dict(
            trial_id=np.tile(trial_id, 3),
            block_id=np.repeat(["all", "block1", "block2"], rp.size),
            rp=np.concatenate([rp, rp_block1, rp_block2]),
            name=exp.sub_name,
            grp=exp.group_tag,
        )
    )
    rwd_df.append(df)

rwd_df = pd.concat(rwd_df, ignore_index=True)
mab_subjects.GroupData().save(rwd_df, "reward_prob")

In [ ]:
import seaborn as sns
import pandas as pd
from neuropy import plotting
import mab_subjects
import matplotlib.pyplot as plt

df = mab_subjects.GroupData().reward_prob.latest

fig = plotting.Fig(5, 2, fontsize=10)

for g, grp in enumerate(["unstruc", "struc"]):
    ax = fig.subplot(fig.gs[g])
    df_grp = df[df["grp"] == grp]

    sns.lineplot(
        data=df_grp,
        x="trial_id",
        y="rp",
        hue="block_id",
        palette="Set2",
        ax=ax,
        lw=1.4,
        err_kws=dict(edgecolor="none"),
        errorbar="se",
    )
    ax.set_title(f"Reward in {grp} sessions")
    ax.set_ylabel("Reward Probability")
    ax.set_xticks([1, 50, 100])
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
    ax.set_ylim(0.45, 0.7)

plt.show()